In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
import torchvision

# from sklearn import tree, svm
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split

import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = .5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
# from lutils.fin.autoencoder import Normalizer

In [2]:
np.set_printoptions(precision=8, suppress=True)

In [3]:
# %matplotlib inline
%matplotlib qt

In [5]:
n_epochs = 40 #100   # number of epochs to run
batch_size = 8  # size of each batch
seq_size = 32

In [6]:
class FutureTrendEncoderLayer(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=32, num_layers=2, dropout=0.2):
        super(FutureTrendEncoderLayer, self).__init__()
        
        self.encoder_lstm = nn.Sequential(
            nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
        ) # batch_size hidden_layer_size * seq_size
        
    def forward(self, x): # 8 32 1
        lstm_out, (h_n, c_n) = self.encoder_lstm(x)
        return lstm_out, h_n # 8 32 32, 2 8 32

class FutureTrendDecoderLayer(nn.Module):
    def __init__(self, input_size=33, hidden_layer_size=32, num_layers=2, dropout=0.2):
        super(FutureTrendDecoderLayer, self).__init__()
        
        self.lstm = nn.Sequential(
            nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
        )
        
        self.decoder_dense = nn.Sequential(
            nn.Linear(1024, 1),
        )
    
    def forward(self, x, state): # 8 32 1, 2 8 32
        batch_size = x.shape[0]
        
        context = state[-1].repeat(x.shape[1], 1, 1).permute(1, 0, 2) # permute(1, 0, 2)  1, 8, 32  8, 32, 32
    
        out = torch.cat((x, context), 2) # 32, 8, 48
        out, (h_n, c_n) = self.lstm(out) # 8 32 32

        return self.decoder_dense(out.reshape(batch_size, -1))

class FutureTrendModule(nn.Module):
    def __init__(self):
        super(FutureTrendModule, self).__init__()
        
        self.encoder_layer = FutureTrendEncoderLayer().to(0)
        self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
    def forward(self, x): # 8 32 1
        encoded, state = self.encoder_layer(x)
        out = self.decoder_layer(x, state)
        return out

In [7]:
# class FutureTrendEncoderLayer(nn.Module):
#     def __init__(self, input_size=1, hidden_layer_size=32, num_layers=4, dropout=0.2):
#         super(FutureTrendEncoderLayer, self).__init__()
        
        
        
#         self.encoder_lstm = nn.Sequential(
#             nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
#         ) # batch_size hidden_layer_size * seq_size
        
#     def forward(self, x): # 8 32 1
#         rnn, h_n = self.encoder_lstm(x)
#         return rnn, h_n # 8 32 32, 2 8 32

# class FutureTrendDecoderLayer(nn.Module):
#     def __init__(self, input_size=2, hidden_layer_size=32, num_layers=4, dropout=0.2):
#         super(FutureTrendDecoderLayer, self).__init__()
        
#         self.lstm = nn.Sequential(
#             nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
#         )
        
#         self.decoder_dense = nn.Sequential(
#             nn.Linear(1024, 1),
#         )
    
#     def forward(self, x, state): # 8 32 1, 2 8 32
#         batch_size = x.shape[0]
        
# #         context = state[-1].repeat(x.shape[1], 1, 1).permute(1, 0, 2) # permute(1, 0, 2)  1, 8, 32  8, 32, 32
        
#         context = state[-1].reshape(batch_size, 32, 1)
        
#         out = torch.cat((x, context), 2) # 32, 8, 48
#         out, h_n = self.lstm(out) # 8 32 32

#         return self.decoder_dense(out.reshape(batch_size, -1))

# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         self.encoder_layer = FutureTrendEncoderLayer().to(0)
#         self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
#     def forward(self, x): # 8 32 1
#         encoded, h_n = self.encoder_layer(x)
#         out = self.decoder_layer(x, h_n)
#         return out, h_n

In [8]:
# class FutureTrendEncoderLayer(nn.Module):
#     def __init__(self, input_size=1, hidden_layer_size=32, num_layers=2, dropout=0.2):
#         super(FutureTrendEncoderLayer, self).__init__()
        
#         self.encoder_lstm = nn.Sequential(
#             nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
#         ) # batch_size hidden_layer_size * seq_size
        
#     def forward(self, x): # 8 32 1
#         rnn, h_n = self.encoder_lstm(x)
#         return rnn, h_n # 8 32 32, 2 8 32

    
# class FutureTrendAttention(nn.Module):
#     def __init__(self):
#         super(FutureTrendAttention, self).__init__()
    
#     def forward(self, decoder_state, encoder_states):
        
#         prob = nn.functional.softmax(encoder_states[-1] + decoder_state, dim=-1)
#         context = torch.sum(prob.unsqueeze(-1) * encoder_states[-1].unsqueeze(-1), 1)
        
#         return prob, context
        
    
# class FutureTrendDecoderLayer(nn.Module):
#     def __init__(self, input_size=1, hidden_layer_size=32, num_layers=2, dropout=0.2):
#         super(FutureTrendDecoderLayer, self).__init__()
        
# #         self.lstm = nn.Sequential(
# #             nn.GRUCell(input_size=input_size, hidden_size=hidden_layer_size)
# #         )
#         self.decoder_lstm = nn.Sequential(
#             nn.GRU(input_size=input_size, hidden_size=hidden_layer_size, num_layers=num_layers, batch_first=True) # , dropout=dropout
#         )
        
#         self.attention = FutureTrendAttention()
        
#         self.decoder_dense = nn.Sequential(
#             nn.Linear(32, 1),
#         )
    
#     def forward(self, x, state): # 8 32 1, 2 8 32
#         batch_size = x.shape[0]

#         lstm, hn = self.decoder_lstm(x)
#         prob, context = self.attention(hn[-1], state)
        
#         return self.decoder_dense(prob), prob, context

# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         self.encoder_layer = FutureTrendEncoderLayer().to(0)
#         self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
#     def forward(self, x): # 8 32 1
#         encoded, h_n = self.encoder_layer(x)
#         out, prob, context = self.decoder_layer(x, h_n)
#         return out, h_n, prob, context

In [9]:
# class FutureTrendEncoderLayer(nn.Module):
#     def __init__(self):
#         super(FutureTrendEncoderLayer, self).__init__()
        
#         self.linear = nn.Linear(32, 256)
    
#         self.multihead_attn = nn.MultiheadAttention(8, 4, batch_first=True)
        
#     def forward(self, x): # 8 32 1
#         batch_size = x.size(0)
        
#         out = self.linear(x.reshape(batch_size, -1))
#         out = out.reshape(batch_size, 32, -1)
        
#         attn_output, attn_output_weights = self.multihead_attn(out, out, out) # 8 32 8, 8 32 32
# #         print(attn_output.shape)
# #         print(attn_output_weights.shape)
#         return attn_output, attn_output_weights

    
    
# class FutureTrendDecoderLayer(nn.Module):
#     def __init__(self):
#         super(FutureTrendDecoderLayer, self).__init__()
        
#         self.linear = nn.Linear(32, 256)
    
#         self.decoder_dense = nn.Sequential(
#             nn.Linear(512, 1),
#         )
    
#     def forward(self, x, attn): # 8 32 1, 8 32 8
#         batch_size = x.shape[0]
#         linear = self.linear(x.reshape(batch_size, -1))
#         out = linear.reshape(batch_size, 32, -1)
        
#         out = torch.cat((out, attn), 0)
        
#         return self.decoder_dense(out.reshape(batch_size, -1)), linear

# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         self.encoder_layer = FutureTrendEncoderLayer().to(0)
#         self.decoder_layer = FutureTrendDecoderLayer().to(0)
                    
#     def forward(self, x): # 8 32 1
#         attn, attn_weights = self.encoder_layer(x)
#         out, linear = self.decoder_layer(x, attn)
#         return out, attn, attn_weights, linear
    

In [10]:
model = FutureTrendModule().to(0)
# model.load_state_dict(torch.load('D:/option/models/model_0619_autoencoder.pt'))

In [11]:
loss_fn = nn.MSELoss().to(0)  # mean square error
# loss_fn = nn.SmoothL1Loss().to(0)
# loss_fn = nn.CrossEntropyLoss().to(0)
# optimizer = optim.Adam(model.parameters(), lr=0.000001, betas=(0.9, 0.98), eps=1e-9)
# optimizer = optim.SGD(model.parameters(), lr = 0.000001, momentum=0.09)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) #

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
lambda1 = lambda epoch: np.power(0.1, epoch) if epoch <= 6 else 1e-9
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

In [12]:
# SAVE_PATH = 'D:/option/datas'
# SAVE_PATH = 'D:/option/l8'

# DATA_PATH = 'l8_1S_1T_mean'
# DATA_PATH = 'l8_10S_1T_mean'
# DATA_PATH = 'l8_30S_1T_mean'
# DATA_PATH = 'l8_30S_5T_mean'
# DATA_PATH = 'l8_60S_10T_mean'
# DATA_PATH = 'l8_30S_7T_mean'
# DATA_PATH = '30S_15T_mean'

# DATA_PATH = '1S_1T'
# DATA_PATH = '5S_1T'
# DATA_PATH = '15S_15T'
DATA_PATH = '30S_15T'

In [13]:
# SAVE_PATH = os.path.join('D:/option/l8_autoencoder/', DATA_PATH)
# SAVE_PATH = os.path.join('D:/option/master/', DATA_PATH)

SAVE_PATH = os.path.join('D:/option/master/diff', DATA_PATH)

In [14]:
class TimeSeriesDataset(Dataset):
    def __init__(self, x, y):
#         x = np.log(x)
#         y = np.log(y)
        x = np.expand_dims(x, 2) # in our case, we have only 1 feature, so we need to convert `x` into [batch, sequence, features] for LSTM
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [15]:
writer = SummaryWriter('runs/log')

In [16]:
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []

step = 0
# training loop
for epoch in range(n_epochs):

#     total_size = 0
    
#     for ix in range(0, len(dfs_x)-20):
    for j, f in enumerate(os.listdir(SAVE_PATH)):
        if not f.startswith('SHFE.rb'):
            continue
        
        store = pd.HDFStore(os.path.join(SAVE_PATH, f), 'r')
#         X = normalize(store['X'].values)
        X = store['X'].values
        y = store['y']['y'].values
        store.close()
        
        split_index = int(y.shape[0] * .8)
        
        data_x_train = X[:split_index]
        data_x_val = X[split_index:]
        data_y_train = y[:split_index]
        data_y_val = y[split_index:]
        
        dataset_train = TimeSeriesDataset(data_x_train, data_y_train)
        dataset_val = TimeSeriesDataset(data_x_val, data_y_val)
        
        train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
        
        model.train()
        total_size = data_y_train.shape[0]
        running_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(train_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
            optimizer.zero_grad()
            
#             y_pred, h_n, prob, context = model(X_batch) # fc1, lstm1, h_n, dropout, encoded, decoded
            y_pred, attn, attn_weights, linear = model(X_batch) # fc1, lstm1, h_n, dropout, encoded, decoded
            loss = loss_fn(y_pred.reshape(-1), y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X_batch.shape[0]
            
            step += X_batch.shape[0]
            
#             if step % 8000 == 0:
            if idx == 1 and j % 4 == 0:
                writer.add_scalar('loss/loss', loss.item(), step)
                writer.add_scalar('loss/total_loss', running_loss / (total_size), step)

                writer.add_histogram('train/01_x', torchvision.utils.make_grid(X_batch).data.cpu(), step)
#                 writer.add_histogram('train/02_fc1', torchvision.utils.make_grid(fc1).data.cpu(), step)
#                 writer.add_histogram('train/03_lstm1', torchvision.utils.make_grid(lstm1).data.cpu(), step)
#                 writer.add_histogram('train/04_h_n', torchvision.utils.make_grid(h_n).data.cpu(), step)
#                 writer.add_histogram('train/05_dropout', torchvision.utils.make_grid(dropout).data.cpu(), step)
                
#                 writer.add_histogram('train/06_encoded', torchvision.utils.make_grid(encoded).data.cpu(), step)
#                 writer.add_histogram('train/07_decoded', torchvision.utils.make_grid(decoded).data.cpu(), step)
                
#                 writer.add_histogram('train/07_prob', torchvision.utils.make_grid(prob).data.cpu(), step)
#                 writer.add_histogram('train/08_context', torchvision.utils.make_grid(context).data.cpu(), step)

                writer.add_histogram('train/02_attn', torchvision.utils.make_grid(attn).data.cpu(), step)
                writer.add_histogram('train/03_attn_weights', torchvision.utils.make_grid(attn_weights).data.cpu(), step)
                writer.add_histogram('train/04_linear', torchvision.utils.make_grid(linear).data.cpu(), step)

    
                writer.add_histogram('train/09_pred', torchvision.utils.make_grid(y_pred).data.cpu(), step)

#         print('Epoch: [{}/{}], [{}/{}], Loss: {:.8f}'.format(epoch + 1, n_epochs, start+10, total_size, running_loss / (X_batch.shape[0] * i)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, ix, running_loss / (total_size)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size)))
        
        val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
        
        model.eval()
        eval_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(val_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
#             y_pred, h_n, prob, context = model(X_batch) # fc1, lstm1, h_n, dropout, encoded, decoded
            y_pred, attn, attn_weights, linear = model(X_batch) # fc1, lstm1, h_n, dropout, encoded, decoded
            loss = loss_fn(y_pred.reshape(-1), y_batch).cpu().detach().numpy()
            
            eval_loss += loss.item() * y_batch.shape[0]
        eval_loss = eval_loss / data_y_val.shape[0]
        
#         print('eval loss {:.8f}'.format(eval_loss))
        lr = scheduler.get_last_lr()[0]
        print('Epoch: [{}/{}], {}, Loss: {:.8f}, Eval loss {:.8f}, lr:{:.16f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size), eval_loss, lr))
        if eval_loss < best_mse:
            best_mse = eval_loss
            best_weights = copy.deepcopy(model.state_dict())
    
    scheduler.step()
    
# restore model and return best accuracy
model.load_state_dict(best_weights)

Epoch: [1/40], SHFE.rb.2021-04-06.30S.15T.20703.h5, Loss: 0.09513190, Eval loss 0.06628207, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-07.30S.15T.20703.h5, Loss: 0.28667251, Eval loss 0.04139526, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-08.30S.15T.20703.h5, Loss: 0.20733220, Eval loss 0.07147027, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-09.30S.15T.20703.h5, Loss: 0.12185304, Eval loss 0.48043557, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-12.30S.15T.20703.h5, Loss: 0.08974909, Eval loss 0.05144312, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-13.30S.15T.20703.h5, Loss: 0.12853120, Eval loss 0.13957956, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-14.30S.15T.20703.h5, Loss: 0.07672848, Eval loss 0.10595966, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-15.30S.15T.20703.h5, Loss: 0.15393881, Eval loss 0.64676287, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-16.30S.15T.20703.h5, Loss: 0.08884728, Eval loss 0.034457

Epoch: [1/40], SHFE.rb.2021-07-22.30S.15T.20703.h5, Loss: 0.44327325, Eval loss 0.03383216, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-23.30S.15T.20703.h5, Loss: 0.15948236, Eval loss 0.15145572, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-26.30S.15T.20703.h5, Loss: 0.30066139, Eval loss 0.08199762, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-27.30S.15T.20703.h5, Loss: 0.25834428, Eval loss 0.24664173, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-28.30S.15T.20703.h5, Loss: 0.21093230, Eval loss 0.06310276, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-29.30S.15T.20703.h5, Loss: 0.21847510, Eval loss 0.12157296, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-30.30S.15T.20703.h5, Loss: 0.39244704, Eval loss 1.60640500, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-08-02.30S.15T.20703.h5, Loss: 0.42985288, Eval loss 0.06153562, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-08-03.30S.15T.20703.h5, Loss: 0.37168946, Eval loss 0.287190

Epoch: [1/40], SHFE.rb.2021-11-10.30S.15T.20703.h5, Loss: 0.85964106, Eval loss 0.31993277, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-11.30S.15T.20703.h5, Loss: 0.39611479, Eval loss 0.22380647, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-12.30S.15T.20703.h5, Loss: 0.46354597, Eval loss 0.21566832, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-15.30S.15T.20703.h5, Loss: 0.52131598, Eval loss 0.20814291, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-16.30S.15T.20703.h5, Loss: 0.42333277, Eval loss 0.12089466, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-17.30S.15T.20703.h5, Loss: 0.21631802, Eval loss 0.05447910, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-18.30S.15T.20703.h5, Loss: 0.24175842, Eval loss 0.17424243, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-19.30S.15T.20703.h5, Loss: 0.80203133, Eval loss 0.22504399, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-22.30S.15T.20703.h5, Loss: 0.15814717, Eval loss 0.121886

Epoch: [1/40], SHFE.rb.2022-02-28.30S.15T.20703.h5, Loss: 0.11982154, Eval loss 0.07034027, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-01.30S.15T.20703.h5, Loss: 0.16538643, Eval loss 0.04615580, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-02.30S.15T.20703.h5, Loss: 0.17123557, Eval loss 0.05277448, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-03.30S.15T.20703.h5, Loss: 0.29971133, Eval loss 0.07721932, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-04.30S.15T.20703.h5, Loss: 0.17094204, Eval loss 0.07157023, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-07.30S.15T.20703.h5, Loss: 0.46480598, Eval loss 0.69888837, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-08.30S.15T.20703.h5, Loss: 0.55801286, Eval loss 0.16600688, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-09.30S.15T.20703.h5, Loss: 0.26012038, Eval loss 0.05643598, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-10.30S.15T.20703.h5, Loss: 0.20529316, Eval loss 0.122334

Epoch: [1/40], SHFE.rb.2022-06-16.30S.15T.20703.h5, Loss: 0.19697302, Eval loss 0.58926329, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-17.30S.15T.20703.h5, Loss: 0.18111993, Eval loss 0.11322700, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-20.30S.15T.20703.h5, Loss: 0.61777727, Eval loss 0.50202816, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-21.30S.15T.20703.h5, Loss: 0.20334954, Eval loss 0.08705180, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-22.30S.15T.20703.h5, Loss: 0.21784144, Eval loss 0.06331958, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-23.30S.15T.20703.h5, Loss: 0.23696220, Eval loss 0.06795574, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-24.30S.15T.20703.h5, Loss: 0.20380132, Eval loss 0.62181667, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-27.30S.15T.20703.h5, Loss: 0.07635928, Eval loss 0.06092603, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-28.30S.15T.20703.h5, Loss: 0.13555498, Eval loss 0.105272

...

Epoch: [8/40], SHFE.rb.2022-07-13.30S.15T.20703.h5, Loss: 0.37021203, Eval loss 0.51468646, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-14.30S.15T.20703.h5, Loss: 0.17406462, Eval loss 0.21136793, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-15.30S.15T.20703.h5, Loss: 0.33669040, Eval loss 0.37152784, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-18.30S.15T.20703.h5, Loss: 0.22875350, Eval loss 0.07292438, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-19.30S.15T.20703.h5, Loss: 0.38199862, Eval loss 0.25363512, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-20.30S.15T.20703.h5, Loss: 0.21602315, Eval loss 0.15450060, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-21.30S.15T.20703.h5, Loss: 0.21076846, Eval loss 0.06730684, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-22.30S.15T.20703.h5, Loss: 0.13770866, Eval loss 0.10058206, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-07-25.30S.15T.20703.h5, Loss: 0.08621940, Eval loss 0.120043

Epoch: [8/40], SHFE.rb.2022-10-31.30S.15T.20703.h5, Loss: 0.08341908, Eval loss 0.09841892, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-01.30S.15T.20703.h5, Loss: 0.10385875, Eval loss 0.05182853, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-02.30S.15T.20703.h5, Loss: 0.09537649, Eval loss 0.01707202, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-03.30S.15T.20703.h5, Loss: 0.06148246, Eval loss 0.01799080, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-04.30S.15T.20703.h5, Loss: 0.05211310, Eval loss 0.04023914, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-07.30S.15T.20703.h5, Loss: 0.05756678, Eval loss 0.03894308, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-08.30S.15T.20703.h5, Loss: 0.07124767, Eval loss 0.04162227, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-09.30S.15T.20703.h5, Loss: 0.05450972, Eval loss 0.02627825, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2022-11-10.30S.15T.20703.h5, Loss: 0.16798304, Eval loss 0.031174

Epoch: [8/40], SHFE.rb.2023-02-16.30S.15T.20703.h5, Loss: 0.13190057, Eval loss 0.02497280, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-17.30S.15T.20703.h5, Loss: 0.07419656, Eval loss 0.04291989, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-20.30S.15T.20703.h5, Loss: 0.05610774, Eval loss 0.01887574, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-21.30S.15T.20703.h5, Loss: 0.04259412, Eval loss 0.02680962, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-22.30S.15T.20703.h5, Loss: 0.08974290, Eval loss 0.05109172, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-23.30S.15T.20703.h5, Loss: 0.09111954, Eval loss 0.02164436, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-24.30S.15T.20703.h5, Loss: 0.11654520, Eval loss 0.05947107, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-27.30S.15T.20703.h5, Loss: 0.08318481, Eval loss 0.02297283, lr:0.0000000000010000
Epoch: [8/40], SHFE.rb.2023-02-28.30S.15T.20703.h5, Loss: 0.06577693, Eval loss 0.010396

Epoch: [9/40], SHFE.rb.2021-05-12.30S.15T.20703.h5, Loss: 0.40426400, Eval loss 0.11823060, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-13.30S.15T.20703.h5, Loss: 1.35364759, Eval loss 0.25357038, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-14.30S.15T.20703.h5, Loss: 0.67357962, Eval loss 0.76985852, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-17.30S.15T.20703.h5, Loss: 0.59703514, Eval loss 0.08142116, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-18.30S.15T.20703.h5, Loss: 0.23353671, Eval loss 0.17944469, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-19.30S.15T.20703.h5, Loss: 0.64049880, Eval loss 0.80391888, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-20.30S.15T.20703.h5, Loss: 0.51783861, Eval loss 0.63084520, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-21.30S.15T.20703.h5, Loss: 0.39880968, Eval loss 0.94541178, lr:0.0000000000010000
Epoch: [9/40], SHFE.rb.2021-05-24.30S.15T.20703.h5, Loss: 1.11703036, Eval loss 0.288404

KeyboardInterrupt: 

In [17]:
model.load_state_dict(best_weights)

<All keys matched successfully>

In [18]:
y_pred

tensor([[ 0.0378],
        [ 0.5671],
        [ 0.4276],
        [-0.2576],
        [-0.0089],
        [ 0.2174],
        [ 0.1183],
        [ 0.3963]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [19]:
# torch.save(model.state_dict(), 'D:/option/models/model_0621_autoencoder_attention.pt')

In [20]:
# model.load_state_dict(torch.load('D:/option/models/model_0612_01_l8_30S_5T_mean_lstm4.pt'))

In [21]:
# model.load_state_dict(torch.load('D:/option/models/model_0613_all_lstm4.pt'))
# model.load_state_dict(torch.load('D:/option/models/model_0616_all_lstm.pt'))

In [22]:
device = 0

In [23]:
# TEST_DIR = os.path.join('D:/option/l8_autoencoder', '%s_test' % DATA_PATH)
TEST_DIR = os.path.join('D:/option/master/diff', '%s_test' % DATA_PATH)

In [30]:
def plot_file(p):
    store = pd.HDFStore(p, 'r')
    df_x = store['X']
    df_y = store['y']
    last_price = store['last_price']
    store.close()
    
    X_test = df_x.values
    y_test = df_y['y'].values
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
#         y_pred, h_n, prob, context = model(X_batch.to(0)) # fc1, lstm1, h_n, dropout, encoded, decoded
        y_pred, attn, attn_weights, linear = model(X_batch.to(device)) # fc1, lstm1, h_n, dropout, encoded, decoded
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)
#     y_inverse = df_y['y'] * df_y['std'] + df_y['mu'] # (x*self.sd) + self.mu
#     y_inverse = scaler.inverse_transform(df_y['y'].values)
#     print(y_inverse)
#     print(last_price['last_price'].values)

    fig, ax1 = plt.subplots()
    
#     print(y_pred)
#     print(np.log(y_test))
    ax1.plot(y_pred, label='pred', color='b', marker='x')
    ax1.plot(y_test, label='real', color='r', marker='.')

    ax2 = ax1.twinx()
    ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
#     ax2.plot(y_inverse, color='y', label='inserse_price', marker='2')


    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.show()

In [31]:
%matplotlib qt

In [32]:
# plot_file(os.path.join('D:/option/', 'SHFE_rb_mu.h5'))
# plot_file(os.path.join('D:/option/', 'SHFE_rb_mu_day.h5'))

# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-10.41384.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.30S.15T.20704.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.10S.3T.20704.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.15S.15T.20704.h5'))
plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.30S.15T.20704.h5'))

In [30]:
%matplotlib inline

In [31]:
# TEST_DIR = os.path.join('D:/option/', '%s_test' % DATA_PATH)
TEST_DIR = os.path.join('D:/option/master/diff', '%s_test' % DATA_PATH)

In [33]:
for f in os.listdir(TEST_DIR):
    print(f)
    store = pd.HDFStore(os.path.join(TEST_DIR, f), 'r')
    df_x = store['X']
    df_y = store['y']
    last_price = store['last_price']
    store.close()
    
    X_test = df_x.values
    y_test = df_y['y'].values
#     y_inverse = df_y['y'] * df_y['std'] + df_y['mu']
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
        y_pred = model(X_batch.to(0)) # , fc1, lstm1, h_n, dropout
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)
    
#     plt.plot(y_pred, label='pred')
#     plt.plot(y_test, label='real')
#     plt.legend()
#     plt.savefig(os.path.join('D:/option/', '%s_test_fig_lstm4' % DATA_PATH, '%s.jpg' % f), dpi=160)
#     plt.close()
    fig, ax1 = plt.subplots()

    ax1.plot(y_pred, label='pred', color='b', marker='x')
    ax1.plot(y_test, label='real', color='r', marker='.')

    ax2 = ax1.twinx()
    ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
#     ax2.plot(y_inverse.values, color='y', label='inserse_price', marker='2')
    
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.savefig(os.path.join('D:/option/master/diff', '%s_test_fig' % DATA_PATH, '%s.jpg' % f), dpi=160)
    plt.close()
    

SHFE.rb.2023-05-04.15S.15T.20704.h5
SHFE.rb.2023-05-05.15S.15T.20704.h5
SHFE.rb.2023-05-08.15S.15T.20704.h5
SHFE.rb.2023-05-09.15S.15T.20704.h5
SHFE.rb.2023-05-10.15S.15T.20704.h5
SHFE.rb.2023-05-11.15S.15T.20704.h5
SHFE.rb.2023-05-12.15S.15T.20704.h5
SHFE.rb.2023-05-15.15S.15T.20704.h5
SHFE.rb.2023-05-16.15S.15T.20704.h5
SHFE.rb.2023-05-17.15S.15T.20704.h5
SHFE.rb.2023-05-18.15S.15T.20704.h5
SHFE.rb.2023-05-19.15S.15T.20704.h5
SHFE.rb.2023-05-22.15S.15T.20704.h5
SHFE.rb.2023-05-23.15S.15T.20704.h5
SHFE.rb.2023-05-24.15S.15T.20704.h5
SHFE.rb.2023-05-25.15S.15T.20704.h5
SHFE.rb.2023-05-26.15S.15T.20704.h5
SHFE.rb.2023-05-29.15S.15T.20704.h5
SHFE.rb.2023-05-30.15S.15T.20704.h5
SHFE.rb.2023-05-31.15S.15T.20704.h5
SHFE.rb.2023-06-01.15S.15T.20704.h5
SHFE.rb.2023-06-02.15S.15T.20704.h5
SHFE.rb.2023-06-05.15S.15T.20704.h5
SHFE.rb.2023-06-06.15S.15T.20704.h5
SHFE.rb.2023-06-07.15S.15T.20704.h5
SHFE.rb.2023-06-08.15S.15T.20704.h5
SHFE.rb.2023-06-09.15S.15T.20704.h5


### l8_30S_15T_mean 0.00028